In [ ]:
import sys
sys.path.append('src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pipeline import ExamScorePipeline, prepare_features
from sklearn.model_selection import train_test_split

# Set random seed
np.random.seed(42)

## 1. Load Processed Data

In [ ]:
# Load the processed dataset (created by run_experiment.py)
df = pd.read_csv('artifacts/processed_data.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

## 2. Prepare Features

In [ ]:
# Identify numeric and categorical features
numeric_features, categorical_features = prepare_features(df)

print(f"Numeric features ({len(numeric_features)}):")
print(numeric_features)

print(f"\nCategorical features ({len(categorical_features)}):")
print(categorical_features)

## 3. Create Train/Test Split

In [ ]:
X = df.drop(columns=['exam_score', 'student_id'], errors='ignore')
y = df['exam_score']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")

## 4. Build and Train Pipeline

In [ ]:
# Create pipeline with default configuration
pipeline = ExamScorePipeline(
    scaler_type='standard',
    encoder_type='onehot',
    model_type='randomforest',
    random_state=42,
    n_estimators=200
)

# Build pipeline
pipeline.build_pipeline(numeric_features, categorical_features)

# Train
print("Training model...")
pipeline.fit(X_train, y_train)
print("✅ Training complete!")

## 5. Evaluate Model

In [ ]:
# Evaluate on test set
metrics = pipeline.evaluate(X_test, y_test)

print("Test Set Performance:")
print(f"  MSE:  {metrics['mse']:.2f}")
print(f"  RMSE: {metrics['rmse']:.2f}")
print(f"  MAE:  {metrics['mae']:.2f}")
print(f"  R²:   {metrics['r2']:.4f}")

## 6. Feature Importance

In [ ]:
# Get feature importance
importance_df = pipeline.get_feature_importance()

if importance_df is not None:
    print("Top 15 Important Features:")
    print(importance_df.head(15))
    
    # Visualize
    plt.figure(figsize=(12, 8))
    top_features = importance_df.head(15)
    sns.barplot(data=top_features, x='importance', y='feature')
    plt.title('Top 15 Feature Importance', fontsize=14, weight='bold')
    plt.xlabel('Importance Score')
    plt.tight_layout()
    plt.show()

## 7. Predictions Visualization

In [ ]:
# Make predictions
y_pred = pipeline.predict(X_test)

# Actual vs Predicted
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5, s=20)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
         'r--', lw=2, label='Perfect Prediction')
plt.xlabel('Actual Exam Score')
plt.ylabel('Predicted Exam Score')
plt.title('Actual vs Predicted Exam Scores', fontsize=14, weight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Residual plot
residuals = y_test - y_pred

plt.figure(figsize=(10, 6))
plt.scatter(y_pred, residuals, alpha=0.5, s=20)
plt.axhline(y=0, color='r', linestyle='--', lw=2)
plt.xlabel('Predicted Exam Score')
plt.ylabel('Residuals')
plt.title('Residual Plot', fontsize=14, weight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Test Different Configurations

In [ ]:
# Test different model configurations
configs = [
    {'name': 'RF-Standard', 'scaler': 'standard', 'model': 'randomforest'},
    {'name': 'RF-MinMax', 'scaler': 'minmax', 'model': 'randomforest'},
    {'name': 'GB-Standard', 'scaler': 'standard', 'model': 'gradientboosting'},
    {'name': 'Linear-Standard', 'scaler': 'standard', 'model': 'linear'},
]

results = []

for config in configs:
    print(f"\nTesting: {config['name']}")
    
    pipe = ExamScorePipeline(
        scaler_type=config['scaler'],
        encoder_type='onehot',
        model_type=config['model'],
        random_state=42,
        n_estimators=100 if config['model'] in ['randomforest', 'gradientboosting'] else None
    )
    
    pipe.build_pipeline(numeric_features, categorical_features)
    pipe.fit(X_train, y_train)
    
    metrics = pipe.evaluate(X_test, y_test)
    
    results.append({
        'Configuration': config['name'],
        'R²': metrics['r2'],
        'MSE': metrics['mse'],
        'MAE': metrics['mae']
    })
    
    print(f"  R² = {metrics['r2']:.4f}, MSE = {metrics['mse']:.2f}")

# Display results
results_df = pd.DataFrame(results)
print("\nComparison Results:")
print(results_df.to_string(index=False))

## 9. Save Model

In [ ]:
# Save the best model
pipeline.save('artifacts/demo_model.joblib')
print("✅ Model saved to artifacts/demo_model.joblib")

## 10. Load and Use Model

In [ ]:
# Load saved model
loaded_pipeline = ExamScorePipeline.load('artifacts/demo_model.joblib')

# Make predictions with loaded model
sample = X_test.iloc[:5]
predictions = loaded_pipeline.predict(sample)

print("Sample Predictions:")
for i, (pred, actual) in enumerate(zip(predictions, y_test.iloc[:5])):
    print(f"  Sample {i+1}: Predicted={pred:.1f}, Actual={actual:.1f}, Error={abs(pred-actual):.1f}")